In [2]:
import cv2
import numpy as np

In [99]:
def resize(img, dsize : tuple[int, int]):
    return cv2.resize(img, dsize)

def rotate(img, angle):
    height, width = img.shape
    rot_mat = cv2.getRotationMatrix2D((height/2, width/2), angle, 1.0)
    imgR = cv2.warpAffine(img, rot_mat, (height, width), flags=cv2.INTER_LINEAR)
    return imgR

def reflect(img, isVertical = False):
    if isVertical == None:
        return img
    if isVertical:
        return img[::-1]
    else:
        return img[:,::-1]

In [97]:
# Template matching NOT GOOD
def findObjects(mainIMG, subIMG):
    scales = [1]
    angles = [x for x in range(0, 1, 5)]
    reflections = [None]

    h, w, _ = subIMG.shape
    outIMG = mainIMG.copy()
    mainIMG_B = cv2.GaussianBlur(cv2.cvtColor(mainIMG, cv2.COLOR_BGR2GRAY), (5,5), 0)
    subIMG_B = cv2.GaussianBlur(cv2.cvtColor(subIMG, cv2.COLOR_BGR2GRAY), (5,5), 0)

    listed = []
    for scale in scales:
        for angle in angles:
            for reflection in reflections:
                match = cv2.matchTemplate(mainIMG_B, reflect(rotate(resize(subIMG_B, (int(h*scale), int(w*scale))), angle), reflection), cv2.TM_CCORR_NORMED)
                (minVal, maxVal, minLoc, (x, y)) = cv2.minMaxLoc(match)
                if (x,y) not in listed:
                    listed.append((x,y))
    for val in listed:
        cv2.rectangle(outIMG, (val[0],val[1]), (val[0] + w, val[1] + h), (0,0,255), 3)
    return outIMG

mainCanvas = cv2.imread("assets/lab7.png")
candyGhost = cv2.imread("assets/candy_ghost.png")
scaryGhost = cv2.imread("assets/scary_ghost.png")
pamkinGhost = cv2.imread("assets/pampkin_ghost.png")

cv2.imshow("IMAGE", findObjects(findObjects(findObjects(mainCanvas, scaryGhost), pamkinGhost), candyGhost))
#cv2.imshow("IMAGE", findImageScaled(findImageScaled(findImageScaled(mainCanvas, scaryGhost), pamkinGhost), candyGhost))

key = cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

In [100]:
def findObjects(mainIMG, target):
    
    outIMG = mainIMG.copy()
    mainIMG_C = cv2.cvtColor(mainIMG.copy(), cv2.COLOR_BGR2GRAY)
    targetCopy = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)

    brisk = cv2.SIFT_create(nfeatures=15000)
    keysTarg, descTarg = brisk.detectAndCompute(targetCopy, None)

    isReflected = False

    while True:
        keys, desc = brisk.detectAndCompute(mainIMG_C, None)

        bf = cv2.BFMatcher(cv2.NORM_L1)
        matches = bf.knnMatch(descTarg, desc, k = 2)

        good = []
        for m, n in matches:
            if m.distance < 0.8 * n.distance:
                good.append(m)
        if len(good) < 20:
            if (not isReflected):
                targetCopy = reflect(targetCopy)
                keysTarg, descTarg = brisk.detectAndCompute(targetCopy, None)
                isReflected = True
                continue
            else: break

        targetPts = np.float32([keysTarg[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        mainPts = np.float32([keys[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

        perspective, _ = cv2.findHomography(targetPts, mainPts, cv2.RANSAC, 6)

        h, w = targetCopy.shape
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        rectanglePts = cv2.perspectiveTransform(pts, perspective)

        m = np.int32(np.sort(np.reshape(rectanglePts.ravel(),(4,2)), axis = 0))
        
        mainIMG_C[m[0][1]:m[3][1], m[0][0]:m[3][0]] = 255
        outIMG = cv2.polylines(outIMG, [np.int32(rectanglePts)], True, (0, 0, 255), 4, cv2.LINE_AA)
        
    return outIMG

mainCanvas = cv2.imread("assets/lab7.png")
candyGhost = cv2.imread("assets/candy_ghost.png")
scaryGhost = cv2.imread("assets/scary_ghost.png")
pamkinGhost = cv2.imread("assets/pampkin_ghost.png")

img = findObjects(mainCanvas, candyGhost)
img = findObjects(img, scaryGhost)
img = findObjects(img, pamkinGhost)
y, x, _ = img.shape

cv2.imshow("IMAGE", cv2.resize(img, (int(x*0.6), int(y*0.6))))

key = cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()